In [26]:
import sys
from collections import defaultdict
import itertools
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
import time
from sklearn import metrics
import numpy as np
np.set_printoptions(threshold=100)
import json
import torchtext.vocab

### Import and load GoogleNews key2vec

In [2]:
import gensim
from gensim.models import KeyedVectors
import gensim.downloader as api

info = api.info()  # show info about available models/datasets
key2vec = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  
weights = key2vec.vectors

In [3]:
weights = key2vec.vectors

### Testing replacement words so that we have minimal overlap

In [4]:
key2vec.most_similar("appalling")

[('disgraceful', 0.7855889201164246),
 ('horrendous', 0.7606023550033569),
 ('deplorable', 0.754478394985199),
 ('atrocious', 0.749769926071167),
 ('shameful', 0.72076416015625),
 ('appaling', 0.7162660360336304),
 ('disgusting', 0.7104288935661316),
 ('abominable', 0.7005642652511597),
 ('totally_unacceptable', 0.6859470009803772),
 ('dreadful', 0.6839314699172974)]

In [5]:
key2vec.most_similar("crummy")

[('lousy', 0.8046033382415771),
 ('crappy', 0.7873436212539673),
 ('shitty', 0.6884918212890625),
 ('cruddy', 0.67264723777771),
 ('mediocre', 0.6025679707527161),
 ('horrid', 0.5934531688690186),
 ('awful', 0.5819990038871765),
 ('bad', 0.5677820444107056),
 ('miserable', 0.566582202911377),
 ('god_awful', 0.5625608563423157)]

In [6]:
key2vec.most_similar("okay")

[('alright', 0.8877537250518799),
 ('ok', 0.8567795753479004),
 ('OK', 0.7831767797470093),
 ('yeah', 0.6638473272323608),
 ('allright', 0.6537948846817017),
 ('hey', 0.603424608707428),
 ('say_Hey_feller', 0.5872976779937744),
 ('anyway', 0.5856851935386658),
 ('anyways', 0.5801851153373718),
 ('maybe', 0.5797765851020813)]

In [7]:
key2vec.most_similar("decent")

[('Decent', 0.7122883796691895),
 ('good', 0.6837348341941833),
 ('respectable', 0.6686230301856995),
 ('decently', 0.6438794136047363),
 ('mediocre', 0.6006860733032227),
 ('terrific', 0.5998380184173584),
 ('nice', 0.5993332266807556),
 ('solid', 0.5919034481048584),
 ('middling', 0.5665558576583862),
 ('excellent', 0.5584126710891724)]

In [27]:
key2vec.most_similar("beautiful")

[('gorgeous', 0.8353004455566406),
 ('lovely', 0.8106936812400818),
 ('stunningly_beautiful', 0.7329413890838623),
 ('breathtakingly_beautiful', 0.7231341600418091),
 ('wonderful', 0.6854087114334106),
 ('fabulous', 0.6700063943862915),
 ('loveliest', 0.6612576246261597),
 ('prettiest', 0.6595001816749573),
 ('beatiful', 0.6593325138092041),
 ('magnificent', 0.6591403484344482)]

### Neural Model

In [28]:
SAVE_PATH = 'models/model_'
NUM_EPOCHS = 1000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# all the one-hot embedding etc. has to be handled here, rather in data generation,
# because we have to know which word indice go to which real words in order to do the embedding.

#--------------- Model --------------
class Neural_Net(nn.Module):
    def __init__(self, input_size, output_size, layer_lens = [25,40,50], nonlins = [nn.Tanh(), nn.ReLU()], drop_freqs= []):
        super(Neural_Net, self).__init__()

        self.input_size = input_size
        self.output_size = output_size

        self.layers = nn.ModuleList()
        self.act_ftns = nonlins
        self.dropouts = []

        curr_layer_len = input_size
        for l in layer_lens:
            self.layers.append(nn.Linear(curr_layer_len, l, bias = True))
            curr_layer_len = l
        self.layers.append( nn.Linear(curr_layer_len,self.output_size))

        for f in drop_freqs:
             self.dropouts.append(nn.Dropout(f))
               

    def forward(self, x):
        import itertools
        components = list(itertools.zip_longest(self.layers,self.act_ftns, self.dropouts))
        
        for layer, act_ftn, dropout in components:
            #init components and forward through
            if layer != None:
                x = layer(x)
            if act_ftn != None:
                x = act_ftn(x)
            if dropout != None:
                x = dropout(x)
        x = nn.Softmax()(x)
        return x

#------------------ Loss & Optimizer --------------------
def cross_entropy_loss(y_true, y_hat):
    return torch.mean(torch.sum(- y_true * torch.log(y_hat), 1))

In [12]:
def train(model, optimizer, loss_fn, training_data, num_epochs):
    prev_loss = -100000000
    
    for epoch in range(num_epochs):
        
        x = Variable(torch.Tensor(training_data[0]))
        y = Variable(torch.Tensor(training_data[1]))

        y_hat = model(x)

        optimizer.zero_grad()
        loss = loss_fn(y,y_hat)
        print(epoch, loss.item(), end='\r')

        loss.backward()
        optimizer.step()       
        
        if epoch > 350:
            dev_y_hat = predict(model, dev_x)
            dev_loss = cross_entropy(dev_y,dev_y_hat)
            if prev_loss < dev_loss:
                break
            else:
                prev_loss = loss

In [9]:
#-------------- Metrics ---------------------
def cross_entropy(y_true,y_hat,  eps=1e-15):
    return -(y_true * np.log(y_hat)).sum(axis=1).mean()

# accuracy returns proportion of max(preds) in max(targets)
def accuracy(pred, targets):
    top_targs = (np.max(targets,axis=1)[:,None] == targets) #set indexes with top values as true
    top_preds = (np.max(pred,axis=1)[:,None] == pred) #set indexes with top values as true
    correct_preds = np.sum(top_targs&top_preds,axis=1) #counts intersections of True
    total_top_preds = np.sum(top_preds,axis=1)
    accuracy_ratio = correct_preds/total_top_preds
    return accuracy_ratio.mean()

In [10]:
#read data into variables
synthetic_labels = eval(open('synthetic_fig_5_labels.json','r').read())
synthetic_priors = eval(open('irony/synthetic_prior_states.json').read())

utters = {1:"terrible",2:"bad",3:"neutral",4:"good",5:"amazing"}
states = [1,2,3,4,5]

#function remodified to provide an embedding for utterance passed in. 
#Name will be updated accordingly
def build_one_hot_utter(u):    
    return weights[key2vec.vocab[u].index]

def build_context(c,priors):
    priors_dict = priors[c]
    state_priors = [priors_dict[s] for s in states]
    return state_priors

In [31]:
#Turn json data file into input/output matrices we can work with
def inp_out(data, priors):
    x = []
    y = []
    for c, vals in data.items():
        context_prior = build_context(c,priors)
        for utter, state in vals.items():
            one_hot_u = build_one_hot_utter(utters[utter])
            x.append(np.concatenate((context_prior,one_hot_u),axis=None))
            y.append(state)            
    x = np.array(x)
    y = np.array(y)
    return x,y

In [33]:
#predict on data using a trained model
def predict(model, data):
    x = Variable(torch.Tensor(data))
    
    y_hat = model(x).data.numpy()
    row_sums = y_hat.sum(axis=1)
    y_hat = y_hat / row_sums[:, np.newaxis]
    return y_hat

#prints out evaluation metrics
def evaluate(y,y_hat):
    ce = cross_entropy(y,y_hat)
    acc = accuracy(y_hat,y)
    mse = metrics.mean_squared_error(y_hat,y)
    print("ce: ", ce)
    print("acc: ", acc)
    print("mse: ", mse)

In [15]:
#turn json data into matrices
x,y = inp_out(synthetic_labels, synthetic_priors)

#split for training
split_1 = int(.8 * len(x))
split_2 = int(.9 * len(x))

train_x, train_y = x[:split_1], y[:split_1]
dev_x, dev_y = x[split_1:split_2], y[split_1:split_2]
test_x, test_y = x[split_2:], y[split_2:]

In [16]:
#this grid search will automatically choose the model with the lowest cross entropy

final_model = None
final_model_ce = 1000000

loss_fn = cross_entropy_loss
for learning_rate in [1e-2]:
    for nonlin in [[nn.Tanh(),nn.ReLU()]]:
        for num_units in [[70,80,90]]:
            model = Neural_Net(train_x.shape[1],train_y.shape[1],layer_lens=num_units,
                              nonlins=nonlin).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=.0001)
            train(model,optimizer, loss_fn, (train_x,train_y), num_epochs=NUM_EPOCHS)
            y_hat = predict(model,train_x)
            evaluate(train_y,y_hat)
            
            #find optimal model
            ce = cross_entropy(train_y,y_hat)
            if ce < final_model_ce:
                final_model = model
                final_model_ce = ce

/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


ce:  1.0654976555102087
acc:  0.965
mse:  0.00019486643031936158


In [17]:
torch.save(final_model.state_dict(), "irony_models/model.pth")

### Test on held-out data

In [34]:
y_hat = predict(model,test_x)
evaluate(test_y,y_hat)

ce:  0.8945989011238548
acc:  0.98
mse:  0.0012197232282003482


/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


### Compare against entropy of actual data

In [35]:
cross_entropy(test_y,test_y)

0.8689911294155673

### Predict on data gathered from Mechanical Turk

In [20]:
fig_5_labels = eval(open('fig_5_labels.json','r').read())
priors = eval(open('irony/prior_states.json').read())

x_real,y_real = inp_out(fig_5_labels, priors)
y_hat = predict(model,x_real)
evaluate(y_real,y_hat)

ce:  0.9481536461113158
acc:  0.8888888888888888
mse:  0.0007853106827594084


/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


### Compare against entropy of Mechanical Turk data

In [21]:
cross_entropy(y_real,y_real)

0.9346084394220676

### Redefine utterances to test on synonyms

In [36]:
utters = {1:"appalling",2:"crummy",3:"okay",4:"decent",5:"beautiful"}

In [29]:
x_modif,y_modif = inp_out(synthetic_labels, synthetic_priors) 
split_1 = int(.8 * len(x))
split_2 = int(.9 * len(x))

modif_test_x, modif_test_y = x_modif[split_2:], y_modif[split_2:]
modif_y_hat = predict(model, modif_test_x)
evaluate(modif_test_y,modif_y_hat)

ce:  0.8945989011238548
acc:  0.98
mse:  0.0012197232282003482


/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


### Test on held-out data with synonyms

In [23]:
x_real,y_real = inp_out(fig_5_labels, priors)
y_hat = predict(model,x_real)
evaluate(y_real,y_hat)

ce:  0.9481536461113158
acc:  0.8888888888888888
mse:  0.0007853106827594084


/home/brndon/anaconda3/envs/torch/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


### Calculate accuracy for a model that has no idea of what synonyms mean

In [39]:
unknown_words_pred = np.full(shape=y_modif.shape, fill_value=.20)
accuracy(unknown_words_pred,y_modif)

0.2

In [24]:
unknown_words_pred = np.full(shape=y_real.shape, fill_value=.20)
accuracy(unknown_words_pred,y_real)

0.19999999999999993

### Calculate CE for a model that has no idea of what synonyms mean

In [47]:
unknown_words_pred = np.full(shape=y_modif.shape, fill_value=.20)
cross_entropy(y_modif,unknown_words_pred)

1.6094379124340998

In [48]:
unknown_words_pred = np.full(shape=y_real.shape, fill_value=.20)
cross_entropy(y_real,unknown_words_pred)

1.6094379124341007